# Project Milestone 2
> CS-401 - Applied Data Analysis

> *Group Padawan - Fall 2024*

>*Maxime Ducourau*, *Mehdi Zoghlami*, *Léopold Henry*, *Martin Catheland*, *Jean Perbet*

This notebook is an exploratory analysis of the [YouNiverse](https://zenodo.org/records/4650046) dataset, which contains data about **137k** english-speaking [YouTube](https://youtube.com) channels and their **73M** videos, uploaded between **2005** and **2019**. 

We chose to focus on the **gaming** category, which is one of the most popular on YouTube. We will try to adress the following **research questions**:
- What are the most popular games on YouTube ?
- Is there a link between real-world gaming events and releases, and the popularity of games on YouTube ?
- What are the most linked communities in the gaming category ?

Due to the considerable size of the dataset, we pre-filtered the original dataset to only keep the **gaming videos**, their **comments**, the **channels** that have uploaded at least one video in the gaming category and their **time-series**. We also took advantage of it to remove some useless / heavy-weight fields.

This pre-filtering is available in the notebook `prefiltering.ipynb`. Below is a summary of the datasets at our disposal, after pre-filtering.

| File | Description | Fields |
| --- | --- | -- |
| `gaming_videos.tsv` | Videos | `title`, `tags`, `upload_date`, `view_count`, `like_count`, `dislike_count`, `duration`, `channel_id`, `display_id` |
| `gaming_comments.tsv` | Comments | `author`, `video_id`|
| `gaming_channels.tsv` | Channels | `channel_id`, `channel_name`, `subscribers` |
| `gaming_timeseries.tsv` | Channels time-series | `channel_id`, `datetime`, `views`, `delta_views`, `subs`, `delta_subs`, `videos`, `delta_videos` |

We'll divide our analysis into the previously mentioned datasets. Let's first load the libraries and create the constants we will need later.

It is important to notice that we will use the [`polars`](https://pola.rs) library throughout this notebook, which is a blazingly fast DataFrame library implemented in Rust and available in Python. It provided *lazy* operations, which are usefult not to load the entire dataset in memory.

In [1]:
# data frames
import pandas as pd
import polars as pl

# math
import numpy as np

# visualization
import matplotlib.pyplot as plt

# text processing
import string

# progress tracking
from tqdm import tqdm
tqdm.pandas()

# turn off warnings
import warnings
warnings.filterwarnings("ignore")

# data paths
VIDEOS_PATH = "data/youniverse/filtered/gaming_videos.tsv"
CHANNELS_PATH =  "data/youniverse/filtered/gaming_channels.tsv"
TIMESERIES_PATH = "data/youniverse/filtered/gaming_timeseries.tsv"
COMMENTS_PATH =  "data/youniverse/filtered/gaming_comments.tsv"

# random seed
RANDOM_STATE = 1

### Video metadata

Let's dive into the **videos metadata**.

In [ ]:
videos_df = vaex.open("yt_gaming")

#### Statistics

Great ! Now, let's get some insights about our data, starting with the number of gaming videos we have at our disposal.

In [10]:
videos_df.select(pl.len()).collect().item()

We have more than **13.5 million** videos, that's quite a good knowledge base ! Let's get some statistics, such as the mean **number of views** and **duration**.

In [ ]:
int(videos_df.select().mean().collect().item())

In [5]:
int(gaming_df['duration'].mean())

It's interesting to see that, **on average**, between may 2005 and october 2019, the ~136k english channels having more than 10k followers and 10 videos have ~76k views per video, and their videos last ~24 minutes.

Let's try to visualize the distribution of the number of views and the duration of the videos. We'll create a sample of $5\%$ of the data to avoid runtime issues, while still being representative of the whole dataset.

In [6]:
sample_df = gaming_df.sample(frac=0.05, random_state=RANDOM_STATE)
print(f'There are {len(sample_df)} videos in the sample dataset.')

We convert back our `vaex` DataFrame to a `pandas` DataFrame to have more control over visualization.

In [7]:
pandas_sample_df = sample_df.to_pandas_df()

In [8]:
plt.figure(figsize=(10, 6))
plt.hist(pandas_sample_df['duration'], bins=200, range=(0, pandas_sample_df['duration'].quantile(0.95)), color='skyblue', edgecolor='black')
plt.xlabel('Video Duration (seconds)')
plt.ylabel('Count')
plt.title('Distribution of Video Duration');

In [9]:
plt.figure(figsize=(10, 6))
plt.hist(pandas_sample_df['view_count'], bins=10, range=(0, pandas_sample_df['view_count'].quantile(0.95)), color='skyblue', edgecolor='black')
plt.xlabel('Views')
plt.ylabel('Count')
plt.title('Distribution of Views)');

Now, let's visualize the distribution accross time of the **upload date** of our gaming videos.

In [10]:
pandas_sample_df["upload_date"] = pd.to_datetime(pandas_sample_df["upload_date"])
pandas_sample_df['year_month'] = pandas_sample_df['upload_date'].dt.to_period('M')
monthly_counts = pandas_sample_df['year_month'].value_counts().sort_index()

plt.figure(figsize=(10, 6))
monthly_counts.plot(kind='line')
plt.xlim(right=pd.Timestamp('2019-08-31'))
plt.xlabel('Upload Date')
plt.ylabel('Count')
plt.title('Distribution of Upload Dates')
plt.xticks(rotation=45);


It follows the general trend of YouTube, which **increasing uploads** over time.

## Most common tags & titles

Now, let's dive deeper into our `title` and `tags` field. We noticed that the `tags` field is a list of tags, where they're separated by a comma. What would be super interesting to do is extracting all different tags and see which ones are the most common. Since we start working with the whole dataset again, we take our `vaex` DataFrame.

In [11]:
tag_counts = gaming_df['tags'].str.lower().str.split(',').value_counts()
tag_counts.head(15)

Very interesting ! We can already see that `minecraft` is tagged on a huge number of videos.

Let's apply the same methodology to `title` filed, but his time splitting the words by space.

In [12]:
title_counts = gaming_df['title'].str.lower().str.split().value_counts()
title_counts.head(15)

As we can see, this is less representative. We can try to check some famous games in the `tag_counts` to evaluate their interest.

In [13]:
print(f'Number of tags containing "fifa": {tag_counts["fifa"]}')
print(f'Number of tags containing "minecraft": {tag_counts["minecraft"]}')
print(f'Number of tags containing "league of legends": {tag_counts["league of legends"]}')

## Video games dataset

 Now, we will cross our dataset with another one, containing an almost-comprehensive list of more than **41k** unique commercial video games. It is available [here](https://www.kaggle.com/datasets/matheusfonsecachaves/popular-video-games). So as to keep this study manageable, we will only focus on the first **~1k** most popular games, since we assume they represent the vast majority of YouTube gaming videos.
 
Let's load and filter out the dataset.

In [14]:
games_df = pd.read_csv('data/games.csv', index_col=0).drop_duplicates("Title").reset_index(drop=True)
games_df.head(3)

We'll use the `Plays` feature to estimate the popularity of the games, and we'll use a cutoff of **2k** games to keep.

In [15]:
cutoff = 2000

games_df["Plays_Numeric"] = games_df["Plays"].apply(lambda x: float(x.replace('k', '').replace('K', '')) * 1000 if "k" in x or "K" in x else float(x))
games_df = games_df.sort_values(by="Plays_Numeric", ascending=False).drop(columns=["Plays_Numeric"])
games_df = games_df.iloc[:cutoff]

Then, we'll remove game names that are too short and those that are common english names, as they are likely to be noise. We pick the list of common english names [here](https://github.com/dwyl/english-words).

In [16]:
with open("data/words_alpha.txt", "r") as f:
    words = {line.strip() for line in f}

games_df = games_df[(games_df["Title"].str.len() > 4) & ~(games_df["Title"].str.lower().isin(words))].reset_index(drop=True)

Some video games are just specific versions of other games, see for instance the *Tetris* example.

In [17]:
games_df[games_df["Title"].str.contains("Tetris")]

Since we're not interested in specific versions but rather in the game itself, we will remove all games that are a superstring of another game.

In [18]:
games_df = games_df[games_df['Title'].apply(lambda x: not any(other in x for other in games_df['Title'] if x != other))].reset_index(drop=True)

Since the titles and tags are quite noisy and may contain a lot of irrelevant information as well as different names for a given video game (e.g. *LoL* for *League of Legends*, *gta* for *Grand Theft Auto V*, etc.), we searched for a good and effective way to extract the game names from the `title` and `tags` field. 

- We first tried to use **TF-IDF** embeddings, but it did not yield satisfactory results are rare words were given too much importance : a title containing the words *mafia* and *minecraft* would be assigned to the game *mafia ii*, even though it is more likely to be about *minecraft*. 
- We then tried to use **Sentence Transformers** to embed the game names and the titles/tags, and then compute the cosine similarity between them. It did not yield satisfactory results either. 

- What works the best is in fact quite simple. We lowercase and remove punctuation from all video games names, titles and tags, and then we start by looking if a game name is entirely contained in the video title. If it is, we assign the video to this game. If it is not, we look if a game name is entirely contained in one of the tags. If there is only one game name, we assign the video to this game. Otherwise, if there is no game or several games in the tags, we do not assign the video. This way, we can assign a game to **~50%** of the videos.

During **PM3**, we could go further in our analysis by leveraging **LLMs** to acomplish this task for more videos, but this would require a lot of computational power and time. We could also investigate the use of game initials, since it is a commonly used pattern in video tags (e.g. *gta v* for *Grand Theft Auto V*, *lol* for *League of Legends*, *r6* for *Tom Clancy's Rainbow Six Siege*, *cod* for *Call of Duty*, etc.)

 We'll only provide a **Proof-of-Concept** here, as it is quite slow (~1h) to process the whole dataset. We'll only process the previously devised sample (`pandas_sample_df`) of $5\%$ of the data.

In [19]:
def preprocess_name(name: str) -> str:
    """
    Convert to lowercase and remove punctuation from the name.
    
    Args:
        name: str - Name of the game
    
    Returns:
        str - Processed name
    """
    return name.lower().replace(",", " ").translate(str.maketrans('', '', string.punctuation))


pandas_sample_df["title"] = pandas_sample_df["title"].apply(preprocess_name)
pandas_sample_df["tags"] = pandas_sample_df["tags"].apply(preprocess_name)
game_titles = games_df["Title"].apply(preprocess_name).tolist()

In [20]:
def map_to_game(title: str, tags: str) -> str:
    """
    Map the video to a game based on the title and tags.
    
    Args:
        title: str - The title of the video.
        tags: str - List of tags of the video.
        
    Returns:
        str - The name of the game if the video is related to a game, else None.
    """
    for game in game_titles:
        if game in title:
            return game
     
    matched_games = []
    for game in game_titles:
        if game in tags:
            matched_games.append(game)
    if len(matched_games) == 1:
        return matched_games[0]
    else:
        return None

In [21]:
pandas_sample_df["video_game"] = pandas_sample_df.progress_apply(lambda row: map_to_game(row["title"], row["tags"]), axis=1)

100%|██████████| 686015/686015 [01:37<00:00, 7016.04it/s]


In [22]:
print(f'Percentage of classified games over the sample : {100 - (pandas_sample_df["video_game"].isna().sum() / len(pandas_sample_df) * 100):.2f}%')

Percentage of classified games over the sample : 49.68%


Almost half of our sample was assigned a game ! We can now explore what are the top-15 most popular games in our dataset !

In [26]:
pandas_sample_df[pandas_sample_df["video_game"].notna()].value_counts("video_game")[:15]

video_game
minecraft            38384
fortnite             18745
call of duty         18380
league of legends    11799
roblox               10456
grand theft auto     10201
dota 2                7422
super smash bros      4632
final fantasy         4066
world of warcraft     3875
the sims              3783
assassins creed       3694
resident evil         3266
mortal kombat         3195
clash of clans        3120
Name: count, dtype: int64

Unsurprisingly, YouTube is dominated by *minecraft*, *fortnite*, *call of duty*, *league of legends*, ... This concludes our analysis on the gaming videos metadata.